In [1]:
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import simpledialog
import cv2

In [2]:
img = Image.open("sample.jpg")
x,y = img.size
print(f"Total Pixels for The Given Image: {x*y}")

Total Pixels for The Given Image: 786432


In [4]:
def prepare_mask(height, width, pixels_to_reveal, steps):
    total_pixels = height * width
    mask = np.zeros(total_pixels, dtype=int)
    mask[:pixels_to_reveal] = steps  # Control the fade intensity
    np.random.shuffle(mask)
    mask = mask.reshape((height, width))
    return mask

def fade_in_effect(image_path, pixels_to_reveal, steps=30):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        print("Error loading image")
        return

    height, width, _ = img.shape
    mask = prepare_mask(height, width, pixels_to_reveal, steps)

    for current_step in range(1, steps + 1):
        # Calculate opacity level for the current step
        opacity_level = current_step / steps
        fading_mask = np.clip(mask - current_step + 1, 0, 1).astype(np.uint8)

        # Create a black image to fade in from
        black_background = np.zeros_like(img)
        display_img = cv2.addWeighted(img, opacity_level, black_background, 1 - opacity_level, 0)

        # Apply the pixel reveal mask
        display_img *= fading_mask[:, :, np.newaxis]  # Apply mask to all channels

        cv2.imshow('Fading Image', display_img)
        cv2.waitKey(100)  # Delay in ms, adjust for speed of animation

    cv2.waitKey(0)  # Wait indefinitely until a key is pressed
    cv2.destroyAllWindows()

def ask_user_for_pixels():
    root = tk.Tk()
    root.withdraw()  # Hide the main Tk window

    # Ask the user for the number of pixels to reveal
    pixels_to_reveal = simpledialog.askinteger("Input", "How many pixels to display?",
                                               parent=root, minvalue=0, maxvalue=10000000)  # Adjust max value as needed
    root.destroy()

    return pixels_to_reveal

# Example usage
pixels_to_reveal = ask_user_for_pixels()
if pixels_to_reveal is not None:
    fade_in_effect('sample.jpg', pixels_to_reveal)
